# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 13 2022 1:18PM,246317,10,SOTAH0000777,"Nextsource Biotechnology, LLC",Released
1,Sep 13 2022 1:07PM,246316,20,8521787-T,"ACI Healthcare USA, Inc.",Released
2,Sep 13 2022 12:47PM,246314,10,8521208,MedStone Pharma LLC,Released
3,Sep 13 2022 12:47PM,246314,10,8521209,MedStone Pharma LLC,Released
4,Sep 13 2022 12:47PM,246314,10,8521210,MedStone Pharma LLC,Released
5,Sep 13 2022 12:47PM,246314,10,8521211,MedStone Pharma LLC,Released
6,Sep 13 2022 12:47PM,246314,10,8521212,MedStone Pharma LLC,Released
7,Sep 13 2022 12:47PM,246314,10,8521224,MedStone Pharma LLC,Released
8,Sep 13 2022 12:47PM,246314,10,8521225,MedStone Pharma LLC,Released
9,Sep 13 2022 12:47PM,246314,10,8521226,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,246307,Released,4
22,246310,Released,4
23,246314,Released,9
24,246316,Released,1
25,246317,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246307,NaN,4.0,4.0
246310,NaN,NaN,4.0
246314,NaN,NaN,9.0
246316,NaN,NaN,1.0
246317,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246064,0.0,1.0,0.0
246224,0.0,0.0,1.0
246227,35.0,36.0,48.0
246252,0.0,0.0,1.0
246260,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246064,0,1,0
246224,0,0,1
246227,35,36,48
246252,0,0,1
246260,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246064,0,1,0
1,246224,0,0,1
2,246227,35,36,48
3,246252,0,0,1
4,246260,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246064,,1,
1,246224,,,1
2,246227,35,36,48
3,246252,,,1
4,246260,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 13 2022 1:18PM,246317,10,"Nextsource Biotechnology, LLC"
1,Sep 13 2022 1:07PM,246316,20,"ACI Healthcare USA, Inc."
2,Sep 13 2022 12:47PM,246314,10,MedStone Pharma LLC
11,Sep 13 2022 12:41PM,246310,10,Beach Products Inc
15,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation
23,Sep 13 2022 12:39PM,246306,10,Emerginnova
28,Sep 13 2022 12:29PM,246300,19,"AdvaGen Pharma, Ltd"
64,Sep 13 2022 12:16PM,246297,10,Eywa Pharma Inc.
65,Sep 13 2022 12:13PM,246293,10,Ivaoes Animal Health
66,Sep 13 2022 12:00PM,246287,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 13 2022 1:18PM,246317,10,"Nextsource Biotechnology, LLC",,,1
1,Sep 13 2022 1:07PM,246316,20,"ACI Healthcare USA, Inc.",,,1
2,Sep 13 2022 12:47PM,246314,10,MedStone Pharma LLC,,,9
3,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,,,4
4,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,,4,4
5,Sep 13 2022 12:39PM,246306,10,Emerginnova,,,5
6,Sep 13 2022 12:29PM,246300,19,"AdvaGen Pharma, Ltd",,13,23
7,Sep 13 2022 12:16PM,246297,10,Eywa Pharma Inc.,,,1
8,Sep 13 2022 12:13PM,246293,10,Ivaoes Animal Health,,,1
9,Sep 13 2022 12:00PM,246287,10,Eywa Pharma Inc.,,16,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 1:18PM,246317,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 13 2022 1:07PM,246316,20,"ACI Healthcare USA, Inc.",1,,
2,Sep 13 2022 12:47PM,246314,10,MedStone Pharma LLC,9,,
3,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4,,
4,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,4,4,
5,Sep 13 2022 12:39PM,246306,10,Emerginnova,5,,
6,Sep 13 2022 12:29PM,246300,19,"AdvaGen Pharma, Ltd",23,13,
7,Sep 13 2022 12:16PM,246297,10,Eywa Pharma Inc.,1,,
8,Sep 13 2022 12:13PM,246293,10,Ivaoes Animal Health,1,,
9,Sep 13 2022 12:00PM,246287,10,Eywa Pharma Inc.,6,16,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 1:18PM,246317,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 13 2022 1:07PM,246316,20,"ACI Healthcare USA, Inc.",1,,
2,Sep 13 2022 12:47PM,246314,10,MedStone Pharma LLC,9,,
3,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4,,
4,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,4,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 1:18PM,246317,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Sep 13 2022 1:07PM,246316,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Sep 13 2022 12:47PM,246314,10,MedStone Pharma LLC,9.0,NaN,NaN
3,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4.0,NaN,NaN
4,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,4.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 1:18PM,246317,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Sep 13 2022 1:07PM,246316,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Sep 13 2022 12:47PM,246314,10,MedStone Pharma LLC,9.0,0.0,0.0
3,Sep 13 2022 12:41PM,246310,10,Beach Products Inc,4.0,0.0,0.0
4,Sep 13 2022 12:39PM,246307,10,ISDIN Corporation,4.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2955453,35.0,21.0,0.0
15,246227,48.0,36.0,35.0
19,738820,25.0,14.0,0.0
20,492583,7.0,24.0,0.0
21,246064,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2955453,35.0,21.0,0.0
1,15,246227,48.0,36.0,35.0
2,19,738820,25.0,14.0,0.0
3,20,492583,7.0,24.0,0.0
4,21,246064,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35.0,21.0,0.0
1,15,48.0,36.0,35.0
2,19,25.0,14.0,0.0
3,20,7.0,24.0,0.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,15,Released,48.0
2,19,Released,25.0
3,20,Released,7.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,35.0,0.0,0.0,0.0
Executing,21.0,36.0,14.0,24.0,1.0
Released,35.0,48.0,25.0,7.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,35.0,0.0,0.0,0.0
1,Executing,21.0,36.0,14.0,24.0,1.0
2,Released,35.0,48.0,25.0,7.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,35.0,0.0,0.0,0.0
1,Executing,21.0,36.0,14.0,24.0,1.0
2,Released,35.0,48.0,25.0,7.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()